<a href="https://colab.research.google.com/github/Hbus2/Analysis-Projects/blob/main/Updating_Ratings_for_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

haydenbusie_rating_update_path = kagglehub.dataset_download('haydenbusie/rating-update')
haydenbusie_rating_update_cosmetics_path = kagglehub.dataset_download('haydenbusie/rating-update-cosmetics')
haydenbusie_brands_path = kagglehub.dataset_download('haydenbusie/brands')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
brand_data = pd.read_csv('/kaggle/input/brands/Brands 5 or more.csv')

#Open data set
brand_data.head()
total_rows = brand_data.shape[0]
print(f'Total number of rows: {total_rows}')

In [ ]:
# Keep only the specified columns
cleaned_cosmetic_data = brand_data[['id', 'brand', 'name', 'price_sign', 'price', 'currency', 'rating', 'category', 'product_type', 'product_colors']]

# Display the cleaned DataFrame to verify
display(cleaned_cosmetic_data.head())

# Optionally, you can save the cleaned DataFrame to a new CSV file
cleaned_cosmetic_data.to_csv('/kaggle/working/cleaned_cosmetic_data.csv', index=False)

In [ ]:
# Function to format the colors safely using json.loads
def format_colors(color_list):
    try:
        color_data = eval(color_list)  # or use json.loads(color_list) if it's JSON formatted
        return ', '.join([f'colour_name: {color["colour_name"]}' for color in color_data])
    except (ValueError, TypeError):
        return color_list  # Return original if there's an error

# Apply the function to the 'product_colors' column using .loc
cleaned_cosmetic_data.loc[:, 'product_colors'] = cleaned_cosmetic_data['product_colors'].apply(format_colors)

# Save the new DataFrame to a CSV file
cleaned_cosmetic_data.to_csv('/kaggle/working/cleaned_cosmetic_data.csv', index=False)

# Set the maximum number of rows displayed to 15
pd.set_option('display.max_rows', 15)

# Display the new DataFrame to verify
display(cleaned_cosmetic_data)

# Print the total number of rows in the DataFrame
total_rows = cleaned_cosmetic_data.shape[0]
print(f'Total number of rows: {total_rows}')

In [ ]:
cleaned_cosmetic_data.head()

In [ ]:
#Pull unique currency and price signs
unique_currency_signs = cleaned_cosmetic_data[['currency', 'price_sign']].drop_duplicates()

print("Unique Currencies with Price Sign:")
for index, row in unique_currency_signs.iterrows():
    print(f"{row['currency']} ({row['price_sign']})")



# Group by 'currency' and 'price_sign' and count occurrences
currency_price_sign_count = cleaned_cosmetic_data.groupby(['currency', 'price_sign']).size().reset_index(name='count')

# Display the result
display(currency_price_sign_count)


In [ ]:
# 1. Pull all rows where 'rating' is NaN or empty
missing_rating_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['rating'].isna() | (cleaned_cosmetic_data['rating'] == '')]

# Display the rows with missing ratings
print("Rows with missing ratings:")
missing_rating_rows.head()  # Display the first few rows (or use .head(n) to see n rows)

# 2. Count how many missing ratings each brand has
missing_ratings_by_brand = missing_rating_rows.groupby('brand').size().reset_index(name='missing_ratings_count')

# Display the brands with missing ratings and their counts
print("\nBrands with missing ratings and their counts:")
print(missing_ratings_by_brand)

# Calculate the total number of missing ratings
total_missing_ratings = missing_rating_rows.shape[0]

# Print the total number of missing ratings
print(f"\nTotal missing ratings: {total_missing_ratings}")





# Create a DataFrame summarizing the count of non-null and null values for each column
summary = cleaned_cosmetic_data.isnull().agg(['count', 'sum']).T.rename(columns={'count': 'Non-Null Count', 'sum': 'Null Count'})

# Display the summary DataFrame
display(summary)

# If you want to display only columns with null values (ignoring columns with no nulls)
print("\nColumns with missing data:")
print(summary[summary['Null Count'] > 0])



In [ ]:
def fill_missing_category(data: pd.DataFrame):
    # Fill missing values in the 'category' column with values from 'product_type'
    data['category'] = data['category'].fillna(data['product_type'])

    # Display the updated rows with previously missing 'category' values
    updated_rows = data[data['category'] == data['product_type']]
    print("Filled missing category values with product_type names:")
    display(updated_rows[['id', 'name', 'category', 'product_type', 'price', 'rating']].head(10))


# Run the function
fill_missing_category(cleaned_cosmetic_data)

# Save the updated data if needed
output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index=False)
print(f"Cleaned data saved to {output_path}")


In [ ]:
def display_missing_ratings_for_brand(brand_name):
    # Filter rows where 'brand' matches the given brand_name and 'rating' is NaN
    missing_ratings = cleaned_cosmetic_data[(cleaned_cosmetic_data['brand'].str.lower() == brand_name.lower()) & (cleaned_cosmetic_data['rating'].isna())]

    # Display the rows with missing ratings
    if missing_ratings.empty:
        print(f"No missing ratings found for the brand: {brand_name}")
    else:
        display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']])

#Checking brand Dior
brand_to_check = 'dior'
display_missing_ratings_for_brand(brand_to_check)

cleaned_cosmetic_data['rating'] = pd.to_numeric(cleaned_cosmetic_data['rating'], errors='coerce')

#Updating ratings for Dior
updates = {
    740: 4.0, 730: 4.4, 729: 4.4, 728: 4.4, 727: 4.4, 726: 4.4, 725: 4.4, 724: 4.4,
    723: 4.4, 722: 4.4, 721: 4.4, 720: 4.4, 719: 4.4, 718: 4.4, 662: 4.4, 661: 4.4,
    660: 4.7, 659: 4.7, 658: 4.7, 717: 4.6, 716: 4.7, 715: 4.6, 714: 4.5, 713: 4.5,
    712: 4.6, 711: 4.2, 710: 4.5, 709: 4.7, 708: 4.3, 707: 4.5, 706: 4.6, 705: 4.6
}

# Assign ratings using the dictionary
for id_val, rating in updates.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id'] == id_val, 'rating'] = rating

# Additional price update for one row
cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id'] == 719, 'price'] = 20.0

#Cleaned DataFrame
cleaned_cosmetic_data = cleaned_cosmetic_data[['id', 'brand', 'name', 'price_sign', 'price', 'currency', 'rating', 'category', 'product_type', 'product_colors']]


updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(updates.keys())]
display(updated_rows)

#Cleaned DataFrane to csv file
output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index=False)
print(f"Cleaned data saved to {output_path}")


In [ ]:
import pandas as pd

def fill_missing_ratings(data: pd.DataFrame, brand_name: str):
    # Ensure 'rating' column is numeric
    data['rating'] = pd.to_numeric(data['rating'], errors='coerce')

    # Calculate the mean rating for the specified brand where ratings are not missing
    brand_mean = data.loc[
        (data['brand'].str.lower() == brand_name.lower()) & data['rating'].notna(),
        'rating'
    ].mean()

    # Update missing ratings for the specified brand with the calculated mean
    data.loc[
        (data['brand'].str.lower() == brand_name.lower()) & (data['rating'].isna()),
        'rating'
    ] = brand_mean

    # Display the updated rows for confirmation
    updated_rows = data[(data['brand'].str.lower() == brand_name.lower()) & (data['rating'] == brand_mean)]
    print(f"Updated missing ratings for '{brand_name.capitalize()}' with the mean rating: {brand_mean:.2f}")
    display(updated_rows[['id', 'name', 'brand', 'price', 'rating', 'product_type']].sort_values(by='id'))

# Example usage:
# cleaned_cosmetic_data = pd.read_csv('path_to_your_data.csv')  # Uncomment to load your data as needed

# Run the function for the brand 'Smashbox'
fill_missing_ratings(cleaned_cosmetic_data, 'dior')

# Save the updated data if needed
output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index=False)
print(f"Cleaned data saved to {output_path}")


In [ ]:
def display_missing_ratings_for_brands(brands_list):
    for brand_name in brands_list:
        missing_ratings = cleaned_cosmetic_data[
            (cleaned_cosmetic_data['brand'].str.lower() == brand_name.lower()) &
            (cleaned_cosmetic_data['rating'].isna())
        ]

        # Display missing ratings if any, otherwise notify
        if missing_ratings.empty:
            print(f"No missing ratings found for the brand '{brand_name}'")
        else:
            print(f"\nMissing ratings for brand '{brand_name}':")
            display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']])

# Example usage: List of brands to check for missing ratings
brands_to_check = ['pure anada', 'pacifica', 'almay']  # Add more brands to this list as needed
display_missing_ratings_for_brands(brands_to_check)


cleaned_cosmetic_data['rating']=pd.to_numeric(cleaned_cosmetic_data['rating'], errors='coerce')

update = {376:4.6, 254:4.4, 492:3.9, 83:4.5}

for id_val, rating in update.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id']== id_val, 'rating'] = rating

updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys())]
display(updated_rows)


output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")
print(f"Rows Displayed: {updated_rows.shape[0]}")


In [ ]:

#L'oreal ratings
def display_missing_ratings_from_brand(brand_name):
    missing_ratings = cleaned_cosmetic_data[
        (cleaned_Cosmetic_data['brand'].str.lower()==brand_name.lower())&
        (cleaned_cosmetic_data['rating'].isna())
    ]

    if missing_rating.empty:
        print(f"No missing ratings found for the brand {brand_name}")
    else:
        display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']].head(20))

brand_to_check = "l'oreal"
display_missing_ratings_for_brand(brand_to_check)

cleaned_cosmetic_data['rating']=pd.to_numeric(cleaned_cosmetic_data['rating'], errors='coerce')

update = {470:4.2, 419:4.2, 400:4.6, 345:4.6, 216:4.4, 184:4.6, 166:4.5, 164:4.4, 162:4.0, 154:3.7, 142:4.5,
          141:4.3, 134:4.5,
         117:2.0, 106:4.1, 41:4.6}
for id_val, rating in update.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id']== id_val, 'rating'] = rating

updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys())]
display(updated_rows)


output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")
print(f"Rows Displayed: {updated_rows.shape[0]}")


In [ ]:
#Finding missing ratings for benefit
def display_missing_ratings_from_brand(brand_name):
    missing_ratings = cleaned_cosmetic_data[
        (cleaned_Cosmetic_data['brand'].str.lower()==brand_name.lower())&
        (cleaned_cosmetic_data['rating'].isna())
    ]

    if missing_rating.empty:
        print(f"No missing ratings found for the brand {brand_name}")
    else:
        display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']].head(20))

brand_to_check = 'benefit'
display_missing_ratings_for_brand(brand_to_check)

cleaned_cosmetic_data['rating']=pd.to_numeric(cleaned_cosmetic_data['rating'], errors='coerce')

update = {644:4.4, 643:4.4, 641:4.5, 640:4.6, 639:4.7,638:4.4, 637:3.9, 636:4.7, 635:4.4, 634:4.4,
        638:4.4, 637:3.9, 636:4.7, 635:4.4,612:4.5, 611:4.4, 610:4.3, 609:4.3, 608:4.5, 607:4.3, 606:4.5, 605:4.3, 604:4.3,
          633:4.5,603:4.7}

for id_val, rating in update.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id']== id_val, 'rating'] = rating

updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys())]
display(updated_rows)


output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")
print(f"Rows Displayed: {updated_rows.shape[0]}")

In [ ]:
def fill_missing_ratings(data: pd.DataFrame, brand_name: str):

    data['rating'] = pd.to_numeric(data['rating'], errors='coerce')

    # Calculating the mean rating for Benefit
    brand_mean = data.loc[
        (data['brand'].str.lower() == brand_name.lower()) & data['rating'].notna(),
        'rating'
    ].mean()

    # Updating missing ratings
    data.loc[
        (data['brand'].str.lower() == brand_name.lower()) & (data['rating'].isna()),
        'rating'
    ] = brand_mean

    updated_rows = data[(data['brand'].str.lower() == brand_name.lower()) & (data['rating'] == brand_mean)]
    print(f"Updated missing ratings for '{brand_name.capitalize()}' with the mean rating: {brand_mean:.2f}")
    display(updated_rows[['id', 'name', 'brand', 'price', 'rating', 'product_type']].sort_values(by='id'))


fill_missing_ratings(cleaned_cosmetic_data, 'benefit')
#Updating DataFrame
output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index=False)
print(f"Cleaned data saved to {output_path}")


In [ ]:
#smashbox
def display_missing_ratings_from_brand(brand_name):
    missing_ratings = cleaned_cosmetic_data[
        (cleaned_Cosmetic_data['brand'].str.lower()==brand_name.lower())&
        (cleaned_cosmetic_data['rating'].isna())
    ]

    if missing_rating.empty:
        print(f"No missing ratings found for the brand {brand_name}")
    else:
        display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']].head(20))

brand_to_check = 'smashbox'
display_missing_ratings_for_brand(brand_to_check)

cleaned_cosmetic_data['rating']=pd.to_numeric(cleaned_cosmetic_data['rating'], errors='coerce')

update = {560:4.5, 559:4.3, 558:4.2, 557:4.5, 556:4.5, 555:4.5, 554:4.4, 553:4.5, 552:4.4, 551:4.5,
         550:4.1, 549:4.5, 548:4.5, 547:4.0, 546:4.5,545:4.2, 544:4.2,
          524:4.5, 523:4.4, 522:4.5, 521:3.9, 520:4.4,519:4.0, 518:4.3, 517:4.4, 516:4.6, 515:4.2}

for id_val, rating in update.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id']== id_val, 'rating'] = rating

updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys())]
display(updated_rows)


output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")
print(f"Rows Displayed: {updated_rows.shape[0]}")


In [ ]:
def fill_missing_ratings(data: pd.DataFrame, brand_name: str):

    data['rating'] = pd.to_numeric(data['rating'], errors='coerce')


    brand_mean = data.loc[
        (data['brand'].str.lower() == brand_name.lower()) & data['rating'].notna(),
        'rating'
    ].mean()


    data.loc[
        (data['brand'].str.lower() == brand_name.lower()) & (data['rating'].isna()),
        'rating'
    ] = brand_mean


    updated_rows = data[(data['brand'].str.lower() == brand_name.lower()) & (data['rating'] == brand_mean)]
    print(f"Updated missing ratings for '{brand_name.capitalize()}' with the mean rating: {brand_mean:.2f}")
    display(updated_rows[['id', 'name', 'brand', 'price', 'rating', 'product_type']].sort_values(by='id'))


fill_missing_ratings(cleaned_cosmetic_data, 'smashbox')


output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index=False)
print(f"Cleaned data saved to {output_path}")


In [ ]:
def display_and_update_missing_ratings(data: pd.DataFrame, brand_updates: dict):
    # Ensure ratings are numeric
    data['rating'] = pd.to_numeric(data['rating'], errors='coerce')

    # Identify missing ratings
    missing_ratings = data[data['rating'].isna()]

    # Count non-missing ratings for each brand
    brand_rating_counts = data.dropna(subset=['rating']).groupby('brand').size()

    # Filter for brands with 10 or fewer ratings
    brands_with_few_ratings = brand_rating_counts[brand_rating_counts <= 10].index

    # Filter for missing ratings in these brands
    missing_ratings_few = missing_ratings[missing_ratings['brand'].isin(brands_with_few_ratings)]

    # Display missing ratings for brands with 10 or fewer ratings
    print("Missing Ratings for Brands with 10 or Fewer Ratings:")
    display(missing_ratings_few[['id', 'name', 'brand', 'price', 'rating', 'product_type']].sort_values(by='brand'))

    # Apply updates from brand_updates dictionary
    for id_val, rating in brand_updates.items():
        data.loc[(data['id'] == id_val), 'rating'] = rating

    # Capture updated rows for verification and display
    updated_rows = data[data['id'].isin(brand_updates.keys())]

    print("\nUpdated DataFrame with New Ratings:")
    display(updated_rows[['id', 'name', 'brand', 'price', 'rating', 'product_type']].sort_values(by='id'))

    # Return the updated DataFrame
    return data

# Load your data
# cleaned_cosmetic_data = pd.read_csv('path_to_your_data.csv') # Uncomment and load your data as needed

# Example update dictionary: specify {id: new_rating} pairs
brand_updates = {
    504: 4.7, 375: 4.8, 514: 3.5, 198: 4.0, 204: 4.5, 391: 4.1, 256: 2.0, 408: 3.4,
    506: 4.2, 405: 4.1, 98: 4.7, 215: 3.1, 199: 4.1, 190: 3.6, 377: 4.2, 101: 4.6,
    509: 4.6, 211: 4.0, 200: 4.2, 194: 4.5
}

# Run the function to update and retrieve the updated DataFrame
cleaned_cosmetic_data = display_and_update_missing_ratings(cleaned_cosmetic_data, brand_updates)

# Export the updated DataFrame to a CSV file
output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index=False)
print(f"Cleaned data saved to {output_path}")
print(f"Total Rows Updated: {len(brand_updates)}")


In [ ]:
#physicians formula
def display_missing_ratings_from_brand(brand_name):
    missing_ratings = cleaned_cosmetic_data[
        (cleaned_Cosmetic_data['brand'].str.lower()==brand_name.lower())&
        (cleaned_cosmetic_data['rating'].isna())
    ]

    if missing_rating.empty:
        print(f"No missing ratings found for the brand {brand_name}")
    else:
        display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']].head(20))

brand_to_check = 'physicians formula'
display_missing_ratings_for_brand(brand_to_check)

cleaned_cosmetic_data['rating']=pd.to_numeric(cleaned_cosmetic_data['rating'], errors='coerce')

update = {497:4.8, 262:4.4, 259:4.4, 197:3.4}

for id_val, rating in update.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id']== id_val, 'rating'] = rating

updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys())]
display(updated_rows)


output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")
print(f"Rows Displayed: {updated_rows.shape[0]}")

In [ ]:

def display_missing_ratings_from_brand(brand_name):
    missing_ratings = cleaned_cosmetic_data[
        (cleaned_Cosmetic_data['brand'].str.lower()==brand_name.lower())&
        (cleaned_cosmetic_data['rating'].isna())
    ]
    #Displaying  cargo cosmetics missing ratings
    if missing_rating.empty:
        print(f"No missing ratings found for the brand {brand_name}")
    else:
        display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']].head(20))

brand_to_check = 'glossier'
display_missing_ratings_for_brand(brand_to_check)

cleaned_cosmetic_data['rating']=pd.to_numeric(cleaned_cosmetic_data['rating'], errors='coerce')

update = {1004:4.3, 1003:4.1, 1002:4.5, 1001:4.1, 1000:3.9, 999:3.5}

for id_val, rating in update.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id']== id_val, 'rating'] = rating

updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys())]
display(updated_rows)

#Displaying the updated rows
#Exporting cleaned DataFrame to Csv File
output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")
print(f"Rows Displayed: {updated_rows.shape[0]}")



In [ ]:
#e.l.f
def display_missing_ratings_from_brand(brand_name):
    missing_ratings = cleaned_cosmetic_data[
        (cleaned_Cosmetic_data['brand'].str.lower()==brand_name.lower())&
        (cleaned_cosmetic_data['rating'].isna())
    ]
    #Displaying  cargo cosmetics missing ratings
    if missing_rating.empty:
        print(f"No missing ratings found for the brand {brand_name}")
    else:
        display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']].head(20))

brand_to_check = 'e.l.f.'
display_missing_ratings_for_brand(brand_to_check)

cleaned_cosmetic_data['rating']=pd.to_numeric(cleaned_cosmetic_data['rating'], errors='coerce')

update = {496:4.3, 491:4.3, 454:4.6, 450:4.4, 312:3.4}

for id_val, rating in update.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id']== id_val, 'rating'] = rating

updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys())]
display(updated_rows)

#Displaying the updated rows
#Exporting cleaned DataFrame to Csv File
output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")
print(f"Rows Displayed: {updated_rows.shape[0]}")



In [ ]:
#lotus cosmetics usa
def display_missing_ratings_from_brand(brand_name):
    missing_ratings = cleaned_cosmetic_data[
        (cleaned_Cosmetic_data['brand'].str.lower()==brand_name.lower())&
        (cleaned_cosmetic_data['rating'].isna())
    ]
    #Displaying  cargo cosmetics missing ratings
    if missing_rating.empty:
        print(f"No missing ratings found for the brand {brand_name}")
    else:
        display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']].head(20))

brand_to_check = 'lotus cosmetics usa'
display_missing_ratings_for_brand(brand_to_check)

cleaned_cosmetic_data['rating']=pd.to_numeric(cleaned_cosmetic_data['rating'], errors='coerce')

update= {1018:4.6, 1017:4.5, 1016:0.0, 1015:0.0, 1014:0.0, 1013:0.0, 1012:0.0}

for id_val, rating in update.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id']== id_val, 'rating'] = rating

updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys())]
display(updated_rows)

#Displaying the updated rows
#Exporting cleaned DataFrame to Csv File
output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")
print(f"Rows Displayed: {updated_rows.shape[0]}")

In [ ]:
#revlon
def display_missing_ratings_from_brand(brand_name):
    missing_ratings = cleaned_cosmetic_data[
        (cleaned_Cosmetic_data['brand'].str.lower()==brand_name.lower())&
        (cleaned_cosmetic_data['rating'].isna())
    ]
    #Displaying  cargo cosmetics missing ratings
    if missing_rating.empty:
        print(f"No missing ratings found for the brand {brand_name}")
    else:
        display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']].head(20))

brand_to_check = 'revlon'
display_missing_ratings_for_brand(brand_to_check)

cleaned_cosmetic_data['rating']=pd.to_numeric(cleaned_cosmetic_data['rating'], errors='coerce')

update= {390:4.4, 337:4.2, 264:4.4, 193:4.4, 93:4.4, 75:4.4}

for id_val, rating in update.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id']== id_val, 'rating'] = rating

updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys())]
display(updated_rows)

#Displaying the updated rows
#Exporting cleaned DataFrame to Csv File
output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")
print(f"Rows Displayed: {updated_rows.shape[0]}")

In [ ]:
#fenty
def display_missing_ratings_from_brand(brand_name):
    missing_ratings = cleaned_cosmetic_data[
        (cleaned_Cosmetic_data['brand'].str.lower()==brand_name.lower())&
        (cleaned_cosmetic_data['rating'].isna())
    ]
    #Displaying  cargo cosmetics missing ratings
    if missing_rating.empty:
        print(f"No missing ratings found for the brand {brand_name}")
    else:
        display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']].head(20))

brand_to_check = 'fenty'
display_missing_ratings_for_brand(brand_to_check)

cleaned_cosmetic_data['rating']=pd.to_numeric(cleaned_cosmetic_data['rating'], errors='coerce')

update={850:4.4, 849:4.4, 848:4.7, 847:4.4, 780:4.4}


for id_val, rating in update.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id']== id_val, 'rating'] = rating

updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys())]
display(updated_rows)

#Displaying the updated rows
#Exporting cleaned DataFrame to Csv File
output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")
print(f"Rows Displayed: {updated_rows.shape[0]}")


In [ ]:
# Filter rows where the brand is 'Dior'
dior_data = cleaned_cosmetic_data[cleaned_cosmetic_data['brand'].str.lower() == 'dior'.lower()]

# Display the filtered DataFrame
dior_data.head(15)  # or dior_data to see the entire DataFrame


In [ ]:
#covergirl
def display_missing_ratings_from_brand(brand_name):
    missing_ratings = cleaned_cosmetic_data[
        (cleaned_cosmetic_data['brand'].str.lower() == brand_name.lower()) &
        (cleaned_cosmetic_data['rating'].isna())
    ]


    if missing_ratings.empty:
        print(f"No missing ratings found for the brand {brand_name}")
    else:
        display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']].head(20))


brand_to_check = 'covergirl'
display_missing_ratings_from_brand(brand_to_check)


cleaned_cosmetic_data['rating'] = pd.to_numeric(cleaned_cosmetic_data['rating'], errors='coerce')

# Update ratings for Covergirl
update = {
    499: 4.5, 473: 4.3, 457: 4.6, 406: 4.5, 362: 4.4, 348: 4.4,
    342: 4.2, 271: 4.3, 265: 4.3, 261: 4.3, 250: 4.5, 222: 4.5,
    220: 4.0, 202: 4.0, 123: 4.3, 114: 4.0, 113: 4.5, 105: 4.2,
    53: 4.4, 52: 4.4, 40: 4.4
}


for id_val, rating in update.items():

    cleaned_cosmetic_data.loc[
        (cleaned_cosmetic_data['id'] == id_val) &
        (cleaned_cosmetic_data['brand'].str.lower() == 'covergirl'),
        'rating'
    ] = rating


updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys()) &
                                      (cleaned_cosmetic_data['brand'].str.lower() == 'covergirl')]
display(updated_rows)


output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")
print(f"Rows Displayed: {updated_rows.shape[0]}")

Maybelline ratings cleaned


In [ ]:
#maybelline
def display_missing_ratings_from_brand(brand_name):
    missing_ratings = cleaned_cosmetic_data[
        (cleaned_Cosmetic_data['brand'].str.lower()==brand_name.lower())&
        (cleaned_cosmetic_data['rating'].isna())
    ]

    if missing_rating.empty:
        print(f"No missing ratings found for the brand {brand_name}")
    else:
        display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']].head(20))

brand_to_check = 'maybelline'
display_missing_ratings_for_brand(brand_to_check)

cleaned_cosmetic_data['rating']=pd.to_numeric(cleaned_cosmetic_data['rating'], errors='coerce')

update = {477:4.4, 468:4.5, 339:4.9, 225:4.3, 221:4.3, 97:3.9, 85:4.1, 11:4.4, }


for id_val, rating in update.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id']== id_val, 'rating'] = rating

updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys())]
display(updated_rows)


output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")
print(f"Rows Displayed: {updated_rows.shape[0]}")

Cargo Cosmetics Ratings cleaned


In [ ]:
#cargo cosmetics
def display_missing_ratings_from_brand(brand_name):
    missing_ratings = cleaned_cosmetic_data[
        (cleaned_Cosmetic_data['brand'].str.lower()==brand_name.lower())&
        (cleaned_cosmetic_data['rating'].isna())
    ]
    #Displaying  cargo cosmetics missing ratings
    if missing_rating.empty:
        print(f"No missing ratings found for the brand {brand_name}")
    else:
        display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']].head(20))

brand_to_check = 'cargo cosmetics'
display_missing_ratings_for_brand(brand_to_check)

cleaned_cosmetic_data['rating']=pd.to_numeric(cleaned_cosmetic_data['rating'], errors='coerce')

update = {502:4.5, 501:4.3, 500:4.7, 480:4.7, 472:4.3, 433:4.1, 432:4.5, 343:4.5, 212:4.0, }


for id_val, rating in update.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id']== id_val, 'rating'] = rating

#Displaying the updated rows
updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys())]
display(updated_rows)
#Exporting cleaned DataFrame to Csv File
output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")


print(f"Rows Displayed: {updated_rows.shape[0]}")


Annabelle ratings are cleaned


In [ ]:
#anabelle
def display_misssing_ratings_from_brand(brand_name):
    missing_ratings = cleaned_cosmetic_data[
        (cleaned_cosmetic_data['brand'].str.lower()== brand_name.lower())&
        (cleaned_cosmetic_data['rating'].isna())
    ]
#Displaying brands with missing ratings
    if missing_rating.empty:
        print(f"No missing ratings found for the brand {brand_name}")
    else:
        display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']].head(20))

brand_to_check = 'annabelle'
display_missing_ratings_for_brand(brand_to_check)


cleaned_cosmetic_data['rating']= pd.to_numeric(cleaned_cosmetic_data['rating'], errors='coerce')

update = {464:4.1, 399:3.8, 249:4.2, 209:3.5, 207:3.0,}

for id_val, rating in update.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id']== id_val, 'rating'] = rating

#Displaying the updated rows
updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys())]
display(updated_rows)

#Exporting cleaned DataFrame to Csv File
output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")


print(f"Rows Displayed: {updated_rows.shape[0]}")


In [ ]:
#nyx
def display_missing_ratings_from_brand(brand_name):
    missing_ratings = cleaned_cosmetic_data[
        (cleaned_cosmetic_data['brand'].str.lower() == brand_name.lower()) &
        (cleaned_cosmetic_data['rating'].isna())
    ]

    if missing_rating.empty:
        print(f"No missing ratings found for the brand: {brand_name}")
    else:
          display(missing_ratings[['id', 'name','brand', 'price', 'rating', 'product_type']].head(20))
brand_to_check = 'nyx'
display_missing_ratings_for_brand(brand_to_check)

cleaned_cosmetic_data['rating'] = pd.to_numeric(cleaned_cosmetic_data['rating'],errors='coerce')


update = {998:2.3, 997:2.8, 996:3.8, 995:3.6, 994:4.0, 993:3.7, 992:2.4, 991:4.4, 990:3.1, 989:4.1, 987:4.4,
           986:2.0, 985:4.0, 984:3.0, 983:4.3, 982:4.2, 981:3.9,980:4.3, 979:4.5,988:4.6, 969:3.2,
           968:3.9,967:3.6, 966:2.5, 965:3.5, 964:3.1, 963:2.8, 962:4.1, 961:4.3, 960:3.8,
           959:4.3, 958:4.2, 957:4.4, 956:3.9, 955:2.6, 954:3.8, 953:4.7, 952:3.9, 951:2.2, 950:4.4,
            978:4.4, 977:4.1, 976:4.0, 975:4.3, 974:4.4, 973:4.3, 972:4.0, 971:4.3, 970:4.6, 949:4.4,
            948:4.6, 947:3.8, 946:4.2, 945:3.6, 944:3.5, 943:4.3, 942:4.6, 941:4.5,940:4.1, 939:4.3,
            938:4.2, 937:4.3, 936:4.3, 935:4.4, 934:4.5, 933:4.7, 932:4.6, 931:4.1, 930:4.5, 929:3.2,
            928:4.3, 927:4.3, 926:4.6, 924:4.3, 923:4.1, 922:4.3, 921:4.5, 920:4.2, 919:4.3, 918:4.2,
            917:4.1, 916:4.3, 915:4.4, 914:4.3, 913:4.6, 912:4.4, 911:4.2, 910:3.8, 909:4.3, 908:4.4,
            907:4.2, 906:4.2, 905:4.3, 904:4.5, 903:4.3
         }

for id_val, rating in update.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id']== id_val, 'rating'] = rating


updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys())]
display(updated_rows)


output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")


print(f"Rows Displayed: {updated_rows.shape[0]}")

In [ ]:
data_file = 'cleaned_cosmetic_data.csv'  # Adjust the file extension if necessary
df = pd.read_csv(data_file)


nyx_rated = df[(df['brand'] == 'nyx') & (df['rating'].notnull())]

# Count the number of rows
count_nyx_rated = nyx_rated.shape[0]

print(f'Number of rows with a rating for the brand NYX: {count_nyx_rated}')

In [ ]:
def fill_missing_ratings_by_product_type(brand_name):
    # Filter the data to include only the specified brand
    brand_data = cleaned_cosmetic_data[cleaned_cosmetic_data['brand'].str.lower() == brand_name.lower()]

    # Iterate through each product_type within the brand
    for product_type in brand_data['product_type'].unique():
        # Filter data for the specific product type
        product_type_data = brand_data[brand_data['product_type'] == product_type]

        # Calculate the mean rating for products that have ratings, rounded to 1 decimal place
        mean_rating = round(product_type_data['rating'].mean(), 1)

        # Fill missing ratings with the rounded mean for that product_type
        cleaned_cosmetic_data.loc[
            (cleaned_cosmetic_data['brand'].str.lower() == brand_name.lower()) &
            (cleaned_cosmetic_data['product_type'] == product_type) &
            (cleaned_cosmetic_data['rating'].isna()), 'rating'
        ] = mean_rating

        print(f"Filled missing ratings for '{product_type}' with average rating: {mean_rating:.1f}")

    print(f"All missing ratings for the brand '{brand_name}' have been filled.")

# Example usage: Fill missing ratings for Clinique
fill_missing_ratings_by_product_type('nyx')

# Display the updated rows for Clinique to verify
updated_clinique_data = cleaned_cosmetic_data[cleaned_cosmetic_data['brand'].str.lower() == 'nyx']
display(updated_clinique_data[['id', 'name', 'brand', 'price', 'rating', 'product_type']])



In [ ]:
def fill_missing_ratings(data: pd.DataFrame, brand_name: str):
    # Ensure 'rating' column is numeric
    data['rating'] = pd.to_numeric(data['rating'], errors='coerce')

    # Calculate the mean rating for the specified brand where ratings are not missing, rounded to 1 decimal place
    brand_mean = round(data.loc[
        (data['brand'].str.lower() == brand_name.lower()) & data['rating'].notna(),
        'rating'
    ].mean(), 1)

    # Update missing ratings for the specified brand with the rounded mean
    data.loc[
        (data['brand'].str.lower() == brand_name.lower()) & (data['rating'].isna()),
        'rating'
    ] = brand_mean


    updated_rows = data[(data['brand'].str.lower() == brand_name.lower()) & (data['rating'] == brand_mean)]
    print(f"Updated missing ratings for '{brand_name.capitalize()}' with the mean rating: {brand_mean:.1f}")
    display(updated_rows[['id', 'name', 'brand', 'price', 'rating', 'product_type']].sort_values(by='id'))


fill_missing_ratings(cleaned_cosmetic_data, 'smashbox')

output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index=False)
print(f"Cleaned data saved to {output_path}")



In [ ]:
#Clinique
def display_missing_ratings_for_brand(brand_name):
    missing_ratings = cleaned_cosmetic_data[
        (cleaned_cosmetic_data['brand'].str.lower() == brand_name.lower()) &
        (cleaned_cosmetic_data['rating'].isna())
    ]

    # Display the rows with missing ratings
    if missing_ratings.empty:
        print(f"No missing ratings found for the brand: {brand_name}")
    else:

        display(missing_ratings[['id', 'name', 'brand', 'price', 'rating', 'product_type']].head(10))


brand_to_check = 'clinique'
display_missing_ratings_for_brand(brand_to_check)


cleaned_cosmetic_data['rating'] = pd.to_numeric(cleaned_cosmetic_data['rating'],errors='coerce')


update = {846:4.5, 845:4.6, 844:4.2, 843:3.5, 842:4.6, 841:4.7, 840:4.5, 839:4.7, 838:4.6, 837:4.3,
    836:4.4, 835:4.5, 834:4.3, 833:4.5, 832:4.5, 831:4.2, 830:4.4, 829:4.1, 828:3.7, 827:4.5, 826:3.4, 825:4.5, 824:4.6,
         823:4.6, 822:4.5, 821:4.5, 820:4.0, 819:4.2, 818:4.4, 817:4.1, 816:4.5, 815:4.7, 814:4.6, 813:3.5,
          812:4.3, 811:4.3, 810:3.7, 809:4.2, 808:4.5, 807:4.3, 806:4.2, 805:4.0,
         804:0.0,803:4.4,}

for id_val, rating in update.items():
    cleaned_cosmetic_data.loc[cleaned_cosmetic_data['id']== id_val, 'rating'] = rating


updated_rows = cleaned_cosmetic_data[cleaned_cosmetic_data['id'].isin(update.keys())]
display(updated_rows)

output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index = False)
print(f"cleaned data saved to {output_path}")

print(f"Rows Displayed: {updated_rows.shape[0]}")

In [ ]:
def fill_missing_ratings(data: pd.DataFrame, brand_name: str):

    data['rating'] = pd.to_numeric(data['rating'], errors='coerce')


    brand_mean = data.loc[
        (data['brand'].str.lower() == brand_name.lower()) & data['rating'].notna(),
        'rating'
    ].mean()

    # Update missing ratings for the specified brand with the calculated mean
    data.loc[
        (data['brand'].str.lower() == brand_name.lower()) & (data['rating'].isna()),
        'rating'
    ] = brand_mean


    updated_rows = data[(data['brand'].str.lower() == brand_name.lower()) & (data['rating'] == brand_mean)]
    print(f"Updated missing ratings for '{brand_name.capitalize()}' with the mean rating: {brand_mean:.2f}")
    display(updated_rows[['id', 'name', 'brand', 'price', 'rating', 'product_type']].sort_values(by='id'))


fill_missing_ratings(cleaned_cosmetic_data, 'clinique')

output_path = '/kaggle/working/cleaned_cosmetic_data.csv'
cleaned_cosmetic_data.to_csv(output_path, index=False)
print(f"Cleaned data saved to {output_path}")


In [ ]:
# Function to calculate and fill missing ratings for a brand based on product type
def fill_missing_ratings_by_product_type(brand_name):
    # Filter the data to include only the specified brand
    brand_data = cleaned_cosmetic_data[cleaned_cosmetic_data['brand'].str.lower() == brand_name.lower()]

    # Iterate through each product_type within the brand
    for product_type in brand_data['product_type'].unique():
        # Filter data for the specific product type
        product_type_data = brand_data[brand_data['product_type'] == product_type]

        # Calculate the mean rating for products that have ratings
        mean_rating = product_type_data['rating'].mean()

        # Fill missing ratings with the mean for that product_type
        cleaned_cosmetic_data.loc[
            (cleaned_cosmetic_data['brand'].str.lower() == brand_name.lower()) &
            (cleaned_cosmetic_data['product_type'] == product_type) &
            (cleaned_cosmetic_data['rating'].isna()), 'rating'
        ] = mean_rating

        print(f"Filled missing ratings for '{product_type}' with average rating: {mean_rating:.2f}")

    print(f"All missing ratings for the brand '{brand_name}' have been filled.")

# Example usage: Fill missing ratings for Clinique
fill_missing_ratings_by_product_type('Clinique')

# Display the updated rows for Clinique to verify
updated_clinique_data = cleaned_cosmetic_data[cleaned_cosmetic_data['brand'].str.lower() == 'clinique']
display(updated_clinique_data[['id', 'name', 'brand', 'price', 'rating', 'product_type']])
